# Building Footprint Segmentation, a DrivenData Challenge

In [20]:
import boto3
bucket_name = 'open-cities'
prefix = f'ai-challenge'
folder = "/train_tier_1"

# required to add to the environment variable : AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY
s3 = boto3.client('s3', endpoint_url='https://data.source.coop')

paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix + folder)

counter_files = 0
for page in pages:
    for obj in page['Contents']:
        #print(obj['Key'])
        counter_files +=1
print(f"number of files : {counter_files}")

number of files : 160


### Automatisation from data loading to tile and mask generation in TIFF format

In [ ]:
import os
from tqdm import tqdm 
local_dir = "../data/open-cities"
# Ensure the local directory exists
os.makedirs(local_dir, exist_ok=True)

# required to add to the environment variable : AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY
s3 = boto3.client('s3', endpoint_url='https://data.source.coop')

paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix + folder)

for page in pages:
    for obj in tqdm(page.get('Contents', [])):
        file_key = obj['Key']
        local_file_path = os.path.join(local_dir, os.path.relpath(file_key, prefix))
        
        # Create any necessary subdirectories
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        
        # Download file
        s3.download_file(bucket_name, file_key, local_file_path)
        #print(f"Downloaded {file_key} to {local_file_path}")

In [1]:
import os
os.environ["CURL_CA_BUNDLE"] = "/etc/ssl/certs/ca-certificates.crt"

from matplotlib import pyplot as plt
import numpy as np
from pprint import pprint

import rasterio
from rasterio.windows import Window
import geopandas as gpd
from pyproj import CRS

from pystac import (Catalog, CatalogType, Item, Asset, Collection)

In [73]:
import os
import rasterio
from rasterio.windows import Window
from rasterio.features import rasterize
from shapely.geometry import box

import geopandas as gpd
from pyproj import CRS
import numpy as np
from tqdm import tqdm

class OpenCities:
    def __init__(self, 
                 cols=None, 
                 win_size=1024,
                 base_dir = "../data/open-cities/train_tier_1",
                 target_dir="../data/open-cities/training_data"):
        
        self.cols = cols  # Preset collections
        self.base_dir = base_dir
        self.location_to_scenes = self._location_to_scenes()
        self.window_size = win_size
        self.target_dir = target_dir
        os.makedirs(self.target_dir + "/images", exist_ok=True)
        os.makedirs(self.target_dir + "/masks", exist_ok=True)
    
    def _location_to_scenes(self):
        """Maps each location to a list of scene IDs, excluding those ending with '-labels'."""
        d = {}
        for key in self.cols.keys():
            for i in self.cols[key].get_all_items():
                if not i.id.endswith('-labels'):  # Exclude items ending with '-labels'
                    d.setdefault(key, []).append(i.id)
        return d
    
    def _normalize_path(self, location, scene_id, file_name, type="image"):
        
        if type=='image':
            return os.path.join(self.base_dir, location, scene_id , file_name)
        elif type=='label':
            return os.path.join(self.base_dir, location, f"{scene_id}-labels" , file_name)
        else:
            return None 
    
    def create_windows(self, src):
        """Creates windows to divide the image into tiles of specified size."""
        tile_size = self.window_size
        n_tiles = (src.width // tile_size, src.height // tile_size)
        windows = [
            Window(i * tile_size, j * tile_size, tile_size, tile_size)
            for i in range(n_tiles[0])
            for j in range(n_tiles[1])
        ]
        return windows

    def save_geochip(self, arr, chip_tfm, save_fn='test', crs='EPSG:4326', dtype='uint8'):
        im = (arr).astype(dtype)

        # check im shape, number of channels and expand into (H,W,C) if needed
        if len(im.shape) == 3: 
            num_ch = im.shape[-1]
        else: 
            num_ch = 1
            im = np.expand_dims(im, -1)
        
        file_path = self.target_dir + "/" + f'{save_fn}.tif'

        with rasterio.open(file_path, 'w', driver='GTiff', 
                                height=im.shape[0], width=im.shape[1],
                                count=num_ch, dtype=im.dtype, crs=crs, transform=chip_tfm, compress='LZW') as dst:
            
            for ch in range(num_ch):
                dst.write(im[:,:,ch], indexes=ch+1) #indexes start at 1

    def display_image_mask(self, img_array, mask_array):
        fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
        ax1.imshow(img_array)
        ax2.imshow(mask_array)
        plt.show()


    def pair_images_to_labels(self):
        """Pairs each image with its label and saves them as geochips."""
        for location, scenes in self.location_to_scenes.items():
            for scene_id in scenes:
                scene_item = self.cols[location].get_item(id=scene_id)

                assert scene_item.assets['image'].href is not None 

                # Construct the image path, normalizing it
                image_path = self._normalize_path(location, scene_id, f'{scene_id}.tif', type="image")
                if not os.path.exists(image_path):
                    print(f"Warning: Image file {image_path} not found, skipping.")
                    continue

                # Open the scene image
                with rasterio.open(image_path) as raster_scene_image:
                    list_windows = self.create_windows(raster_scene_image)
                    list_win_boxes = [
                        box(*rasterio.windows.bounds(w, raster_scene_image.meta["transform"]))
                        for w in list_windows
                    ]

                    # Normalize label path and check if it exists

                    labels_path = self._normalize_path(location, f'{scene_id}' ,f'{scene_id}.geojson', type="label")
                    if not labels_path or not os.path.exists(labels_path):
                        print(f"Warning: Labels file {labels_path} not found, skipping.")
                        continue

                    # Load building footprints (labels)
                    scene_labels_gdf = gpd.read_file(labels_path)
                    
                    for i, win_box in enumerate(tqdm(list_win_boxes)):
                        # Create GeoDataFrame for the current window box
                        win_box_gdf = gpd.GeoDataFrame(geometry=[win_box], crs=raster_scene_image.meta["crs"])
                        win_box_gdf = win_box_gdf.to_crs(CRS.from_epsg(4326))

                        # Spatial join between label footprints and tile boundaries
                        gdf_chip = gpd.sjoin(scene_labels_gdf, win_box_gdf, how='inner')
            
                        # Define shapes for rasterizing
                        burn_val = 255
                        shapes = [(geom, burn_val) for geom in gdf_chip.geometry]

                        if len(shapes) > 0:

                            chip_tfm = rasterio.transform.from_bounds(
                                *win_box_gdf.bounds.values[0], self.window_size, self.window_size
                            )
                            # Rasterize mask with the appropriate transform
                            label_arr = rasterize(
                                shapes, 
                                out_shape=(self.window_size, self.window_size), 
                                transform=chip_tfm, 
                                dtype='uint8'
                            )

                            # Extract image tile and save both image and mask
                            win_arr = raster_scene_image.read(window=list_windows[i])
                            win_arr = np.moveaxis(win_arr,0,2)
                            image_title = f"images/{scene_id}_tile_{i}"
                            mask_title = f"masks/{scene_id}_tile_{i}_mask"

                            self.save_geochip(arr=win_arr, chip_tfm=chip_tfm, save_fn=image_title)
                            self.save_geochip(arr=label_arr, chip_tfm=chip_tfm, save_fn=mask_title)

                            if np.random.rand() < 0.05:
                                self.display_image_mask(img_array=win_arr, mask_array=label_arr)

In [74]:
cols = {cols.id:cols for cols in root_catalog.get_children()}

In [75]:
open_cities = OpenCities(
    cols=cols,
    base_dir = "../data/open-cities/train_tier_1",
    target_dir="../data/open-cities/training_data",
    win_size=1024)

In [ ]:
open_cities.pair_images_to_labels() # Need a lot of Disk space

## Training BuildingFootprint Segmentation Models 

In [1]:
# In a Jupyter notebook or IPython environment, run this in the first cell
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

In [26]:
from torch.utils.data import random_split, DataLoader

def split_pytorch_dataset(data, train_ratio=0.8):
    val_ratio = 1 - train_ratio
    # Calculate the sizes for training and validation
    train_size = int(train_ratio * len(data))
    val_size = len(data) - train_size

    train_dataset, val_dataset = random_split(data, [train_size, val_size])

    return train_dataset, val_dataset

In [32]:
### Data Augmentation Technique 
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose(
    [
        A.Resize(512, 512),
        A.HorizontalFlip(p=0.5),  # Random horizontal flip with 50% probability
        A.VerticalFlip(p=0.5),    # Random vertical flip with 50% probability
        A.RandomRotate90(p=0.5),  # Random 90 degree rotation with 50% probability
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
        ToTensorV2(), 
    ], is_check_shapes=False
)


In [33]:
from datasets import  OpenCities_Building_Dataset

base_dir = "../data/Open-cities"
image_dir = f'{base_dir}/images'
mask_dir = f'{base_dir}/masks'

data = OpenCities_Building_Dataset(
    images_dir=image_dir,
    masks_dir=mask_dir,
    transform=transform,
    filter_invalid_image=True
)

Removing Invalid Samples: 100%|██████████| 3979/3979 [04:50<00:00, 13.72it/s]


In [34]:
train_data , valid_data = split_pytorch_dataset(data, train_ratio=0.8)

In [35]:
len(train_data)

3162

In [36]:
from models import ResNet_UNET
model = ResNet_UNET(in_channels=3,out_channels=2, freeze_backbone=False)

In [37]:
import torch.optim as optim
# Define the optimizer with parameter groups
def get_optimizer(model, lr_backbone=1e-4, lr_upconv=1e-3, weight_decay=1e-5):
    # Collect parameters for the backbone and the upconvs
    backbone_params = []
    upconv_params = []

    for name, param in model.named_parameters():
        if param.requires_grad:  # Only include trainable parameters
            if "upconv" in name:
                upconv_params.append(param)
            else:
                backbone_params.append(param)

    # Define the optimizer with parameter groups
    optimizer = optim.AdamW(
        [
            {"params": backbone_params, "lr": lr_backbone},
            {"params": upconv_params, "lr": lr_upconv},
        ],
        weight_decay=weight_decay,
    )

    return optimizer

# Lambda function for warm-up
def lr_lambda(current_step):
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    return max(
        0.0,
        0.5 * (1.0 + torch.cos(torch.pi * (current_step - warmup_steps) / (total_steps - warmup_steps))),
    )


In [38]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_data, batch_size=32, pin_memory=True, shuffle=True)
val_dl = DataLoader(valid_data, batch_size=32, pin_memory=True, shuffle=False)

In [39]:
from losses import DiceLoss, FocalLoss
from metrics import accuracy, f1_score, iou_score, recall, precision
optimizer = get_optimizer(model, lr_backbone=1e-4, lr_upconv=1e-3)
params_opt = {}
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, last_epoch=-1)
params_sc = {}

criterion = FocalLoss(mode="multiclass")
# Define Metrics 
metrics = [accuracy, f1_score, iou_score, recall, precision]

# Early Stopping 
early_stopping_params = {"patience":5, "trigger_times": 0}

In [ ]:
from training import train 
train(
    model=model,
    train_dl=train_dl,
    valid_dl=val_dl,
    optimizer=optimizer,
    scheduler=scheduler,
    params_opt=params_opt,
    params_sc=params_sc,
    loss_fn=criterion,
    metrics=metrics,
    nb_epochs=50,
    experiment_name="ResNet_Unet_OpenCities",
    log_dir="../runs",
    model_dir="../models",
    early_stopping_params=early_stopping_params,
)

Experiment logs are recoded at ../runs/ResNet_Unet_OpenCities_20241119-212643
Epoch 1
----------
Epoch 1 Training completed. Loss: 0.2925

Metrics (Training Phase):

+---------------------------+--------+
|           Metric          | Value  |
+---------------------------+--------+
|          accuracy         | 0.7322 |
|          f1_score         | 0.7322 |
|         iou_score         | 0.6072 |
|        sensitivity        | 0.7322 |
| positive_predictive_value | 0.7322 |
+---------------------------+--------+
Epoch 1 Validation completed. Loss: 0.2609

Metrics (Validation Phase):

+---------------------------+--------+
|           Metric          | Value  |
+---------------------------+--------+
|          accuracy         | 0.7568 |
|          f1_score         | 0.7568 |
|         iou_score         | 0.6388 |
|        sensitivity        | 0.7568 |
| positive_predictive_value | 0.7568 |
+---------------------------+--------+
LOSS train 0.29249433352176635 valid 0.2609354219530084
Sav